In [19]:
import sys, os

pysbf_path = "/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/"
sys.path.insert(0, pysbf_path)
from pysbf import *

from IPython.display import display, Markdown, clear_output
warnings.filterwarnings('ignore')

In [20]:
inFolder = './data/'
outFolder = './output/'
configFolder = pysbf_path + 'pysbf/config/'

In [36]:
name = "u12517"

model = 0   # the main model
suffix = ".%03d" % model

In [22]:
outFolder += "Outputs_"+name+'/'
logFile = outFolder+name+"_model_log.csv"


if exists(logFile):
    with open(logFile, 'r') as f:
        line = f.readline()

    if line.strip("#").strip()[:4] == "uuid":
        old_uuid = line.strip("#").split(":")[1].strip()                   
        if os.path.isdir(outFolder+name+"_"+old_uuid):
            uuid = old_uuid
            params = get_obj_params(open_log_df(logFile))

else:
    print("There is no log-file. \nPlease run elliprof to generate the log file and the relevant plots.")
    print("Elliprof notebook name: \"Run_elliprof_function.ipynb\"")

In [23]:
df_log = open_log_df(logFile)
sky_med = float(df_log.loc["initial_sky_med"].value)

df = open_log_df(logFile)
obj_uuid = df.loc['uuid'].value
objRoot = outFolder+name+'_'+obj_uuid+'/'

X0 = float(df.loc['X0'].value)
Y0 = float(df.loc['Y0'].value)

Config = {} 

Config["name"] = name
Config["inFolder"] = inFolder
Config["objRoot"] = objRoot
Config["configFolder"] = configFolder
Config["X0"] = X0
Config["Y0"] = Y0

SKY = df.loc["sky"].value

In [25]:
hdu_list = fits.open(inFolder + "{}/{}j.fits".format(name, name))
image_data = hdu_list[0].data

TOP = str(int(np.ceil(np.max(image_data))))       # 
BOTTOM = str(int(np.floor(np.min(image_data))))   # ~1000, -2000

In [28]:
THRESHMAX   = TOP # 1850000   # around saturation level
THRESHMIN   = 1000  
 
SCALEAPRADIUS = 3.0

RDNOISE = 200    
FWHM    = 1.4    # same most of the time

In [136]:
IMAGE_IN    =  "resid.000" # "resid" + suffix   ## can be resid + (extended objects + Dmask --> dophot)
IMAGE_VAR = "masked_sej.000"


# IMAGE_VAR   =  "dvar" + suffix  ## renuc factor 2x the one used for sextractor
IMAGE_OUT   =  "dpr" + suffix   ## temp file to use in ds9
OBJECTS_OUT =  "dpo" + suffix   ## the most important
FINISHFILE  =  "dfin" + suffix  ## remove this (for running dophot mulitple times)

In [137]:
## making dpar files

In [138]:
dophot_params = """
= """ + name + """ GO-16262
SKY = """+SKY+"""     Approximate mean sky value in data numbers.
FWHM = """+str(FWHM)+"""             Approx FWHM of objects (pixels) along major axis.
AXIS_RATIO = 0.9       For star objects.  AR=b/a; b=minor axis.
TILT = 90.0            Angle of major axis in degrees; +x=0; +y=90.
EPERDN = 1.0           Electrons per data number.
RDNOISE = """+str(RDNOISE)+"""         Readout noise in electrons.
TOP = """+TOP+"""          Maximum allowed data value in data numbers.
=
AUTOSCALE = 'YES'      Auto-scaling of fitting and masking radii by FWHM.
AUTOSCALEAP = 'YES'    Auto-scaling of star and sky aperture radii by FWHM.
AUTOTHRESH = 'NO'      Auto-scaling of thresholds and bottom.
AUTOBLIT = 'YES'       Auto-scaling of obliteration parameters.
AUTOMEDSCALE = 'YES'   Auto-scaling for median filter box size.
= If autothresh=no take these parameters.
BOTTOM = """+BOTTOM+"""       Lowest allowed data value in data numbers.
THRESHMAX = """+str(THRESHMAX)+"""    Value of maximum threshold.
THRESHMIN = """+str(THRESHMIN)+"""     Value of lowest threshold.
=
PARAMS_DEFAULT = 'paramdefault'   Default parameters file name.
IMAGE_IN = '"""+IMAGE_IN+"""'            Input image name. 
=  If desired a file of extra variance can be specified, which is
=  added to the normal variance of (DATA/eADU)+(RN/eADU)^2.  Note
=  that it must have proper ADU^2 units of variance 
IMAGE_VAR = '"""+IMAGE_VAR+"""'            Input extra variance image
IMAGE_OUT = '"""+IMAGE_OUT+"""'             Output image name.
OBJECTS_OUT = '"""+OBJECTS_OUT+"""'           Output object list file name.
CENTINTMAX = """+TOP+"""              Obliterate if central intensity exceeds this.
TOPSAT = """+TOP+"""                  Ignore pixels with higher counts in saturated stars.
=
OBJTYPE_IN = 'INTERNAL'     Input format: (COMPLETE, INTERNAL, BINARY)
OBJTYPE_OUT = 'COMPLETE'    Output format: (COMPLETE, INTERNAL,INCOMPLETE, BINARY,DAOPHOT)
=   If autoscaleap=yes determine parameters using
SCALEAPRADIUS = """+str(SCALEAPRADIUS)+"""    Size of aperture phot radius in units of FWHM.
END
"""


In [139]:
DPAR_FILE = objRoot + "dpar" + suffix

with open(DPAR_FILE, 'w') as f:
    f.write(dophot_params)

In [140]:
DPAR_FILE 

'./output/Outputs_u12517/u12517_9f43b1726bf2/dpar.000'

In [141]:
cwd = os.getcwd()

In [142]:
os.chdir( '/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/notebooks')
os.getcwd()

'/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/notebooks'

In [143]:
cwd = os.getcwd()
os.chdir(objRoot)
xcmd("cp " + configFolder + "/dophot/paramdefault .")
xcmd("dophot dpar.000 2>&1 | tee dophot.log")
! cat dophot.log
xcmd("rm ./paramdefault")

os.chdir(cwd)


cp /media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/pysbf/config//dophot/paramdefault .

dophot dpar.000 2>&1 | tee dophot.log
 From frame header: NX, NY =   1025  1022  resid.000                               
 From variance header: NX, NY =   1025  1022  masked_sej.000                          
DoPhot iter  1 of 10 at thresh********   Nfound       0 total       0
DoPhot iter  2 of 10 at thresh776046.9   Nfound       0 total       0
DoPhot iter  3 of 10 at thresh337794.0   Nfound       0 total       0
DoPhot iter  4 of 10 at thresh147033.4   Nfound       4 total       4
DoPhot iter  5 of 10 at thresh 64000.0   Nfound      15 total      19
DoPhot iter  6 of 10 at thresh 27857.6   Nfound      27 total      46
DoPhot iter  7 of 10 at thresh 12125.7   Nfound      66 total     112
DoPhot iter  8 of 10 at thresh  5278.0   Nfound     151 total     263
DoPhot iter  9 of 10 at thresh  2297.4   Nfound     319 total     582
DoPhot iter 10 of 10 at thresh  1000.0   Nfound     649 total    123

In [144]:
os.getcwd()

'/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/notebooks'

In [ ]:
 Using Mv = -7.40 for GCs (NOT for JHK)


In [ ]:
negative distance --> fitting first 

Enter new bright end cutoff (22.00):

sigma width of the GC LF in units of mag ~1.2 small objects ~1.3 ~1.4 giant ellipticals

In [ ]:
LIKENEW6 -- IR version 2020
 Enter object list file name (from mkincat): se_lknj.000
 Enter color (B/V/R/I/z/J/H/K/Ks = 0/1/2/3/4/5/6/7/8) of data: 5
 Enter scale ("/p), psf [-]fwhm(p), and [-]dist (Mpc): 0.128, 1.4, 60
 Enter cutoff [-]S/N ratio and T/F for soft bias correction:-4.5


In [ ]:
ptm6 --> modified manually ptm6b